In [1]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import rasterio
import numpy as np
import torch.nn as nn
import torch.optim as optim
from skimage.transform import resize
from tqdm import tqdm
import pandas as pd
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import mapping
from rasterio.features import geometry_mask
import torch.nn.functional as F

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

tif_path = "D:/10_climate_10_590_330/1_tem/2017/tem_2017_01_01.tif"
with rasterio.open(tif_path) as src:
        image_data = src.read(1)  # 读取第一个波段
        transform = src.transform
        crs = src.crs
        bounds = src.bounds
        height, width = src.height, src.width
        profile = src.profile  # 获取文件元数据以便后续导出
    
shp_path = "C:/Users/Administrator/Desktop/边界/中国/china.shp"
shapefile = gpd.read_file(shp_path)
shapefile = shapefile.to_crs(crs)
geometries = [mapping(geometry) for geometry in shapefile.geometry]
mask = geometry_mask(geometries, transform=transform, invert=True, out_shape=(height, width))

Using device: cuda


In [3]:
class SuperResolutionDataset(Dataset):
    def __init__(self, lr_dirs, hr_dirs, transform=None):
        self.lr_files = self._get_all_files(lr_dirs)
        self.hr_files = self._get_all_files(hr_dirs)
        self.transform = transform

    def _get_all_files(self, dirs):
        all_files = []
        for directory in dirs:
            for filename in sorted(os.listdir(directory)):  # 排序保证匹配
                file_path = os.path.join(directory, filename)
                if os.path.isfile(file_path):  # 确保是文件
                    all_files.append(file_path)
        return all_files

    def __len__(self):
        return len(self.lr_files)

    def __getitem__(self, idx):
        lr_path = self.lr_files[idx]
        hr_path = self.hr_files[idx]
        
        with rasterio.open(lr_path) as lr_dataset:
            lr_image = lr_dataset.read(1)  # 读取第一个波段的数据
        with rasterio.open(hr_path) as hr_dataset:
            hr_image = hr_dataset.read(1)  # 读取第一个波段的数据

        # 如果有 transform，应用转换
        if self.transform:
            lr_image = self.transform(lr_image)
            hr_image = self.transform(hr_image)

        else:
        # 如果没有 transform，转换为 PyTorch 张量
            lr_image = torch.from_numpy(lr_image).float()  # (1, H, W)
            hr_image = torch.from_numpy(hr_image).float()  # (1, H, W)

        return lr_image, hr_image
# 定义数据增强和预处理操作
data_transforms = transforms.Compose([
    transforms.ToTensor(),  # 将 NumPy 数组转换为 PyTorch 张量
])

def load_tiff_with_rasterio(file_path):
    with rasterio.open(file_path) as src:
        img = src.read(1)  # 读取第一个波段
    return img

In [4]:
year = list(range(2000,2014))
lr_directories = []
hr_directories = []
for a_index,a in enumerate(year):
    hr_path = f'D:/10_climate_10_590_330/3_tem_min/{a}'
    hr_directories.append(hr_path)
    
    lr_path = f'F:/6_climate_100_59_33/3_tem_min/{a}'
    lr_directories.append(lr_path)
dataset_train = SuperResolutionDataset(lr_dirs=lr_directories, hr_dirs=hr_directories, transform=data_transforms)
dataloader_train = DataLoader(dataset_train, batch_size=8, shuffle=True, num_workers=0)

year = list(range(2014,2019))
lr_directories = []
hr_directories = []
for a_index,a in enumerate(year):
    hr_path = f'D:/10_climate_10_590_330/3_tem_min/{a}'
    hr_directories.append(hr_path)
    
    lr_path = f'F:/6_climate_100_59_33/3_tem_min/{a}'
    lr_directories.append(lr_path)
dataset_val = SuperResolutionDataset(lr_dirs=lr_directories, hr_dirs=hr_directories, transform=data_transforms)
dataloader_val = DataLoader(dataset_val, batch_size=8, shuffle=True, num_workers=0)

year = list(range(2019,2024))
lr_directories = []
hr_directories = []
for a_index,a in enumerate(year):
    hr_path = f'D:/10_climate_10_590_330/3_tem_min/{a}'
    hr_directories.append(hr_path)
    
    lr_path = f'F:/6_climate_100_59_33/3_tem_min/{a}'
    lr_directories.append(lr_path)
dataset_test = SuperResolutionDataset(lr_dirs=lr_directories, hr_dirs=hr_directories, transform=data_transforms)
dataloader_test = DataLoader(dataset_test, batch_size=8, shuffle=True, num_workers=0)

In [5]:
def train_model(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, targets in tqdm(dataloader):
        inputs  = inputs.float().to(device)
        targets = targets.float().to(device)

        inputs[torch.isnan(inputs)] = 0
        targets[torch.isnan(targets)] = 0

        assert not torch.isnan(inputs).any(), "Input contains NaN values"
        assert not torch.isnan(targets).any(), "Target contains NaN values"
        
        optimizer.zero_grad()
        outputs_up = model(inputs)

        loss = criterion(outputs_up, targets).to(device)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

def validate_model(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for inputs, targets in tqdm(dataloader):
            inputs = inputs.float().to(device)
            targets =  targets.float().to(device)
            inputs[torch.isnan(inputs)] = 0
            targets[torch.isnan(targets)] = 0

            outputs = model(inputs)

            loss = criterion(outputs, targets)
            running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss


In [6]:
import torch
import torch.nn as nn

class ChannelAttention(nn.Module):
    def __init__(self, num_features, reduction=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv = nn.Sequential(
            nn.Conv2d(num_features, num_features // reduction, 1, padding=0, bias=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(num_features // reduction, num_features, 1, padding=0, bias=True),
            nn.Sigmoid()
        )

    def forward(self, x):
        y = self.avg_pool(x)
        y = self.conv(y)
        return x * y

class RCAB(nn.Module):
    def __init__(self, num_features, reduction):
        super(RCAB, self).__init__()
        self.body = nn.Sequential(
            nn.Conv2d(num_features, num_features, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(num_features, num_features, 3, padding=1),
            ChannelAttention(num_features, reduction)
        )

    def forward(self, x):
        return x + self.body(x)

class ResidualGroup(nn.Module):
    def __init__(self, num_features, num_blocks, reduction):
        super(ResidualGroup, self).__init__()
        self.body = nn.Sequential(
            *[RCAB(num_features, reduction) for _ in range(num_blocks)],
            nn.Conv2d(num_features, num_features, 3, padding=1)
        )

    def forward(self, x):
        return x + self.body(x)

class RCAN(nn.Module):
    def __init__(self, num_features=64, num_groups=10, num_blocks=20, reduction=16, scale=10, in_channels=1, out_channels=1):
        super(RCAN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, num_features, 3, padding=1)
        
        self.body = nn.Sequential(
            *[ResidualGroup(num_features, num_blocks, reduction) for _ in range(num_groups)],
            nn.Conv2d(num_features, num_features, 3, padding=1)
        )
        
        self.upsample = nn.Sequential(
            nn.Conv2d(num_features, num_features * (scale ** 2), 3, padding=1),
            nn.PixelShuffle(scale)
        )
        
        self.conv2 = nn.Conv2d(num_features, out_channels, 3, padding=1)

    def forward(self, x):
        x = self.conv1(x)
        res = self.body(x)
        res += x
        x = self.upsample(res)
        x = self.conv2(x)
        return x

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = RCAN(num_features=64, num_groups=10, num_blocks=20, reduction=16, scale=10, in_channels=1, out_channels=1).to(device)

learning_rate = 0.0001
criterion = nn.L1Loss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-4)

num_epochs = 100
best_loss = float('inf')

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f'Model is training on: {next(model.parameters()).device}')
    train_loss = train_model(model, dataloader_train, criterion, optimizer, device)
    val_loss = validate_model(model, dataloader_val, criterion, device)
    print(f'Train Loss: {train_loss:.4f} | Validation Loss: {val_loss:.4f}')
    
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), 'G:/16_ECMWF/13_DL_modle/4_climate_590_330/RCAN/RCAN_tem_min.pth')
        print('Model saved!')


cuda
Epoch 1/100
Model is training on: cuda:0


100%|██████████| 229/229 [00:45<00:00,  5.03it/s]


Train Loss: 60.5663 | Validation Loss: 12.9837
Model saved!
Epoch 2/100
Model is training on: cuda:0


100%|██████████| 229/229 [00:44<00:00,  5.19it/s]


Train Loss: 6.0881 | Validation Loss: 5.1287
Model saved!
Epoch 3/100
Model is training on: cuda:0


...

Train Loss: 0.6420 | Validation Loss: 0.6769
Epoch 99/100
Model is training on: cuda:0


100%|██████████| 229/229 [00:40<00:00,  5.63it/s]


Train Loss: 0.7411 | Validation Loss: 0.8018
Epoch 100/100
Model is training on: cuda:0


100%|██████████| 229/229 [00:40<00:00,  5.60it/s]


Train Loss: 0.7788 | Validation Loss: 0.6011
Model saved!
